<!DOCTYPE html>
<img src="https://sigarra.up.pt/feup/pt/WEB_GESSI_DOCS.download_file?p_name=F-370784536/logo_cores_oficiais.jpg" alt="img" width=200 align="right">
<table>   
  <tr>     
    <td><b><font size = "4">Faculty of Engineering - University of Porto <br><br> Masters in Data Science and Engineering 2022/2023</font></td></b>    
  </tr>
  <tr>
    <td> <font size = "4"><b> Analysis of Complex Data <br><br> 
    Students:</b></font><br>
      <font size="3">
        Danilo Brandao (201109207@fe.up.pt)<br>
        Heitor Lira (202109217@fe.up.pt)<br>
        Wagner Ceulin (202101494@fe.up.pt)
      </font>
  </tr>
</table>

<br>
<center><font size = "5"><b>Investigation of Political Communication Networks </b></font><br></center>

In [ ]:
!pip install -q --upgrade pip
!pip install -q --upgrade networkx
!pip install --force-reinstall -q "scipy==1.8.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 69.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.1 which is incompatible.


In [ ]:
import pandas as pd
import sqlite3 as sql
import json
import networkx as nx
import numpy as np
import statistics as stats
from operator import itemgetter
from scipy import sparse
import scipy
import matplotlib.pyplot as plt

In [ ]:
# check modules
# specific versions are required to avoid conflict
!python --version
print("Scipy: ",scipy.__version__)  # should be 1.8.0
print("NetworkX: ",nx.__version__) # should be >= 2.8
print("Numpy: ",np.__version__)

Python 3.8.16
Scipy:  1.8.0
NetworkX:  3.0
Numpy:  1.21.6


In [ ]:
# accessing data collected from Twitter
!git clone https://github.com/danilogb/tweetgrab.git

Cloning into 'tweetgrab'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 24 (delta 10), reused 20 (delta 6), pack-reused 0
Unpacking objects: 100% (24/24), done.


In [ ]:
cnx = sql.connect('tweetgrab/data/tweets.db')

In [ ]:
df = pd.read_sql_query("SELECT * FROM tweets", cnx)

In [ ]:
df.head()

,tweet_id,text,created_at,location,user_id,username,mentions,hashtags
0,1615126936459657217,RT @AndreJanonesAdv: Quando Bolsonaro assumiu ...,2023-01-16 23:20:43+00:00,null,1584183166029512704,fabiomelosyaho1,"[{""screen_name"": ""AndreJanonesAdv"", ""name"": ""A...",[]
1,1615126936572788738,A Trump y a Bolsonaro le dan un pucherazo a Pu...,2023-01-16 23:20:43+00:00,null,1145721431059292163,Francis16893780,[],[]
2,1615126937768456192,RT @AndreJanonesAdv: Quando Bolsonaro assumiu ...,2023-01-16 23:20:44+00:00,null,204973112,ARCS1974,"[{""screen_name"": ""AndreJanonesAdv"", ""name"": ""A...",[]
3,1615126938024312832,RT @GeorgMarques: Antes do desgoverno Bolsonar...,2023-01-16 23:20:44+00:00,null,1584311891559587842,LeleW340,"[{""screen_name"": ""GeorgMarques"", ""name"": ""Geor...",[]
4,1615126938611326976,RT @TheArtNewspaper: As rioters in Brasília st...,2023-01-16 23:20:44+00:00,null,14544450,Johanna_Moore,"[{""screen_name"": ""TheArtNewspaper"", ""name"": ""T...",[]


# Data Preprocessing

In [ ]:
def extract_mentions(mention):
  mention_list = []
  if json.loads(mention):
    for user in json.loads(mention):
      id = user['id']
      username = user['screen_name']
      mention_list.append((id,username))
      # mention_list.append(username)
  else:
    pass
  return mention_list

In [ ]:
df.mentions = df.mentions.apply(lambda x: extract_mentions(x))

In [ ]:
# check up to how many people are mentioned in tweets
set([len(mention) for mention in df.mentions])

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

In [ ]:
# remove unwanted information from hashtags column
def extract_hashtags(hashtags):
  hashtags_list = []
  if json.loads(hashtags):
    for hashtag in json.loads(hashtags):
      hashtags_list.append(hashtag['text'])
  else:
    pass
  return hashtags_list

In [ ]:
df.hashtags = df.hashtags.apply(lambda x: extract_hashtags(x))

In [ ]:
df.head()

,tweet_id,text,created_at,location,user_id,username,mentions,hashtags
0,1615126936459657217,RT @AndreJanonesAdv: Quando Bolsonaro assumiu ...,2023-01-16 23:20:43+00:00,null,1584183166029512704,fabiomelosyaho1,"[(1198004545, AndreJanonesAdv)]",[]
1,1615126936572788738,A Trump y a Bolsonaro le dan un pucherazo a Pu...,2023-01-16 23:20:43+00:00,null,1145721431059292163,Francis16893780,[],[]
2,1615126937768456192,RT @AndreJanonesAdv: Quando Bolsonaro assumiu ...,2023-01-16 23:20:44+00:00,null,204973112,ARCS1974,"[(1198004545, AndreJanonesAdv)]",[]
3,1615126938024312832,RT @GeorgMarques: Antes do desgoverno Bolsonar...,2023-01-16 23:20:44+00:00,null,1584311891559587842,LeleW340,"[(40548874, GeorgMarques)]",[]
4,1615126938611326976,RT @TheArtNewspaper: As rioters in Brasília st...,2023-01-16 23:20:44+00:00,null,14544450,Johanna_Moore,"[(38681183, TheArtNewspaper)]",[]


## Timespan of analyses

It is important to know what and how big is the period in time being analyzed to understand the results at the end.

In [ ]:
df.created_at = [d.split("+")[0] for d in df.created_at]

In [ ]:
from datetime import datetime

start = datetime.strptime(min(df.created_at), "%Y-%m-%d %H:%M:%S")
end = datetime.strptime(max(df.created_at), "%Y-%m-%d %H:%M:%S")
print(f"The period in time being analyzed goes from {start} to {end}.")
print(f"This is a total of {end.day - start.day} days, \
{end.hour - start.hour} hours, {end.minute - start.minute} minutes and \
{end.second - start.second} seconds.")

The period in time being analyzed goes from 2023-01-16 23:20:43 to 2023-01-16 23:59:59.
This is a total of 0 days, 0 hours, 39 minutes and 16 seconds.


### Visualizing interactions

In [ ]:
df2 = df.copy() # make a safety copy of original dataframe
df2 = df2[df2.mentions.astype(bool)]  # remove users who don't mention anyone

# check if there are missing users
df2.username.isnull().value_counts()

False    9433
Name: username, dtype: int64

In [ ]:
# Generates main graph object
graph = nx.Graph()

In [ ]:
# Function to get the interactions between the different users
def get_interactions(row):
  user = row['user_id'], row['username']

  # interactions are going to be a set of tuples
  interactions = set()

  # add interactions
  for mention in row['mentions']:
    interactions.add(mention)

  # discard interactions with the user itself and None
  interactions.discard((row['user_id'], row['username']))
  interactions.discard((None, None))

  return user, interactions

In [ ]:
for index, tweet in df2.iterrows():
  user, interactions = get_interactions(tweet)
  user_id, username = user
  tweet_id = tweet['tweet_id']

  for interaction in interactions:
    int_id, int_name = interaction
    graph.add_edge(user_id, int_id, tweet_id=tweet_id)

    graph.nodes[user_id]['name'] = username
    graph.nodes[int_id]['name'] = int_name

In [ ]:
print(f"There are {graph.number_of_nodes()} nodes and {graph.number_of_edges()} edges present in the Graph")

There are 8364 nodes and 10214 edges present in the Graph


In [ ]:
degrees = [val for (node, val) in graph.degree()]

print(f"The maximum degree of the Graph is {np.max(degrees)}")   
print(f"The minimum degree of the Graph is {np.min(degrees)}")  

The maximum degree of the Graph is 1302
The minimum degree of the Graph is 1


In [ ]:
print(f"The average degree of the nodes in the Graph is {np.mean(degrees):.1f}")  
print(f"The most frequent degree of the nodes found in the Graph is {stats.mode(degrees)}") 

The average degree of the nodes in the Graph is 2.4
The most frequent degree of the nodes found in the Graph is 1


In [ ]:
if nx.is_connected(graph):
    print("The graph is connected")
else:
    print("The graph is not connected")

The graph is not connected


In [ ]:
print(f"There are {nx.number_connected_components(graph)} connected components in the Graph")  

There are 445 connected components in the Graph


In [ ]:
largest_subgraph = max(nx.connected_components(graph), key=len)

largest_components = sorted(nx.connected_components(graph), key=len, reverse=True)
largest_subgraph = graph.subgraph(largest_components[0])

In [ ]:
largest_subgraph.number_of_nodes(), largest_subgraph.number_of_edges()

(7112, 9388)

In [ ]:
if nx.is_connected(largest_subgraph):
    print("The largest subgraph is connected")
else:
    print("The largest subgraph is not connected")

The largest subgraph is connected


In [ ]:
print(f"The average clustering coefficient is {nx.average_clustering(largest_subgraph):.7f} in the largest subgraph.")
print(f"The transitivity of the largest subgraph is {nx.transitivity(largest_subgraph):.7f}.")

The average clustering coefficient is 0.0133522 in the largest subgraph.
The transitivity of the largest subgraph is 0.0003947.


In [ ]:
print(f"The diameter of our Graph is {nx.diameter(largest_subgraph)}")
print(f"The average distance between any two nodes is {nx.average_shortest_path_length(largest_subgraph):.2f}")

The diameter of our Graph is 18
The average distance between any two nodes is 5.41


## Calculate relevant network metrics

In [ ]:
graph_centrality = nx.degree_centrality(largest_subgraph)
max_de = max(graph_centrality.items(), key=itemgetter(1))
graph_closeness = nx.closeness_centrality(largest_subgraph)
max_clo = max(graph_closeness.items(), key=itemgetter(1))
graph_betweenness = nx.betweenness_centrality(largest_subgraph, normalized=True, endpoints=False)
max_bet = max(graph_betweenness.items(), key=itemgetter(1))

In [ ]:
print(f"The node with id {max_de[0]} has a degree centrality of {max_de[1]:.2f} which is the maximum of the Graph.")
print(f"The node with id {max_clo[0]} has a closeness centrality of {max_clo[1]:.2f} which is the maximum of the Graph.")
print(f"The node with id {max_bet[0]} has a betweenness centrality of {max_bet[1]:.2f} which is the maximum of the Graph.")

The node with id 1198004545 has a degree centrality of 0.18 which is the maximum of the Graph.
The node with id 1198004545 has a closeness centrality of 0.30 which is the maximum of the Graph.
The node with id 1198004545 has a betweenness centrality of 0.47 which is the maximum of the Graph.


In [ ]:
node_and_degree = largest_subgraph.degree()
central_node = [1198004545]
central_node_color = 'red'

In [ ]:
central_user = largest_subgraph.nodes[central_node[0]]['name']
print(f"User @{central_user} is the central element found in the network.")
print(f"Check it out: https://twitter.com/{largest_subgraph.nodes[central_node[0]]['name']}")

User @AndreJanonesAdv is the central element found in the network.
Check it out: https://twitter.com/AndreJanonesAdv


In [ ]:
pos = nx.spring_layout(largest_subgraph, k=0.05)

In [ ]:
# label 'relevant' nodes only
labels = {
    n: (largest_subgraph.nodes[n]['name']
        if len(list(nx.all_neighbors(largest_subgraph, n))) > 40  # level of importance
        else '')
    for n in largest_subgraph.nodes
}

In [ ]:
plt.figure(figsize = (20,20))
# nx.draw(largest_subgraph, pos=pos, node_color=range(7112), cmap=plt.cm.PiYG, edge_color="black", linewidths=0.3, node_size=60, alpha=0.6, with_labels=True, labels=labels)
nx.draw(largest_subgraph, pos=pos, node_color=range(7112), cmap=plt.cm.PiYG, edge_color="black", linewidths=0.3, node_size=60, alpha=0.6, with_labels=False)
nx.draw_networkx_nodes(largest_subgraph, pos=pos, nodelist=central_node, node_size=300, node_color=central_node_color)
nx.draw_networkx_labels(largest_subgraph, pos, labels, font_size=16, font_color="whitesmoke")
plt.savefig('graphfinal.png')
plt.show()

Output hidden; open in https://colab.research.google.com to view.

### About network visualizations

"Proper graph visualization is hard, and we highly recommend that people visualize their graphs with tools dedicated to that task".
This is the official recommendation from the NetworkX module documentation.

In other words, the tools used in this work are focused on getting network metrics and exploring technical parameters. The visualization features are lacking, therefore leading to poor and, oftentimes, confusing network graphs.

However, the work developed here was successful in providing and insight in how political conversations unfold in online environments. It was especially useful in identifying active and relevant actors in these communities.